<a href="https://colab.research.google.com/github/tomonari-masada/course2025-sml/blob/main/12_dimensionality_reduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dimensionality reduction （次元圧縮, 次元削減）
* 下記Webページを参考にした。
  * https://scikit-learn.org/stable/auto_examples/decomposition/plot_faces_decomposition.html

In [ ]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.decomposition import PCA, NMF, FastICA

%config InlineBackend.figure_format = 'retina'

## データの取得
* Sign MNISTというデータセットを使う。(The American Sign Language letter database)
  * https://www.kaggle.com/datamunge/sign-language-mnist
* あらかじめダウンロードしておき、Google Driveに保存しておく。
  * Canvas LMSの「プランナークラスのデータファイル」にも置いてあります。

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/data/sign_mnist_train.csv')
test = pd.read_csv('/content/drive/MyDrive/data/sign_mnist_test.csv')

In [ ]:
train.head()

In [ ]:
plt.imshow(train.loc[0,'pixel1':].values.reshape(28,28), cmap=plt.cm.gray);

In [ ]:
# Setting the label and the feature columns
y_train = train.loc[:,'label'].values
X_train = train.loc[:,'pixel1':].values
y_test = test.loc[:,'label'].values
X_test = test.loc[:,'pixel1':].values

In [ ]:
print(X_train.shape, X_test.shape)

* 25種類あるらしい。

In [ ]:
for label, count in zip(*np.unique(y_train, return_counts=True)):
  print(label, count)

## データの前処理
* scikit-learnのドキュメンテーションにあるやり方に倣った。
  * まずtraining set全体での各ピクセルの平均値を引き算し・・・
  * 次に各画像内でのローカルな平均値を引き算する。

In [ ]:
# global centering
train_mean = X_train.mean(axis=0)
X_train_centered = X_train - train_mean
X_test_centered = X_test - train_mean

In [ ]:
# local centering
X_train_centered = X_train_centered - X_train_centered.mean(axis=1).reshape(-1, 1)
X_test_centered = X_test_centered - X_test_centered.mean(axis=1).reshape(-1, 1)

In [ ]:
plt.imshow(X_train_centered[0,:].reshape(28,28), cmap=plt.cm.gray);

## PCAによる可視化
* 第3主成分までを取り、データ集合を可視化。

In [ ]:
start = time.time()
pca = PCA(n_components=3)
components = pca.fit_transform(X_train_centered)
print(f"Duration: {time.time() - start} seconds")

In [ ]:
total_var = pca.explained_variance_ratio_.sum() * 100

fig = px.scatter_3d(
    components, x=0, y=1, z=2, color=y_train,
    title=f'Total Explained Variance: {total_var:.2f}%',
    labels={'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'},
    width=800, height=800,
    opacity=0.7,
    template='plotly_dark',
    color_continuous_scale=px.colors.sequential.Plasma,
    range_color=[0, 25],
)
fig.update_layout(
    margin=dict(l=40, r=40, b=40, t=40),
    width=900,
    height=500
)
fig.show();

## 次元圧縮手法によるデータの再構成(reconstruction)
* 様々な次元圧縮手法を使って元のデータを低次元空間にマッピングする。
* そしてそれを元の空間へ戻すことでreconstructする。
  * reconstructionは、次元圧縮手法ごとに異なる。
  * inverse_transformというメソッドを呼べばよい。

* 画像を複数描画する関数を定義しておく。

In [ ]:
def plot_gallery(title, images, n_col=3, n_row=3, cmap=plt.cm.gray):
  plt.figure(figsize=(2. * n_col, 2.26 * n_row))
  plt.suptitle(title, size=16)
  for i, comp in enumerate(images):
    plt.subplot(n_row, n_col, i + 1)
    vmax = max(comp.max(), -comp.min())
    plt.imshow(
        comp.reshape(28, 28),
        cmap=cmap,
        interpolation="nearest",
        vmin=-vmax,
        vmax=vmax,
    )
    plt.xticks([])
    plt.yticks([])
  plt.subplots_adjust(0.01, 0.05, 0.99, 0.93, 0.04, 0.);

In [ ]:
plot_gallery("sign mnist", X_train_centered[:9], n_row=3, n_col=3)

### PCAの主成分を可視化する
* PCAでは、principal componentsの線型結合によって元の画像を再構成することになる。


In [ ]:
n_components = 30
estimator = PCA(n_components=n_components)
estimator.fit(X_train_centered)

In [ ]:
n_col = 6

In [ ]:
n_row = n_components // n_col + (n_components % n_col != 0)
plot_gallery(
    f"{n_components} PCA components",
    estimator.components_[:n_components],
    n_row=n_row,
    n_col=n_col,
)

* 8枚ほどの画像について、オリジナルと再構成を比較してみる。

In [ ]:
n_recon_images = 8

In [ ]:
indices = np.random.randint(X_test.shape[0], size=n_recon_images)
plot_gallery("original test data", X_test_centered[indices], n_row=1, n_col=n_recon_images)
X_test_recon = estimator.inverse_transform(estimator.transform(X_test_centered))
plot_gallery("reconstructed test data", X_test_recon[indices], n_row=1, n_col=n_recon_images)

# 課題20250705
* PCA以外の次元圧縮手法について、同様に、元の画像の再構成を実施してみよう。
  * https://scikit-learn.org/stable/auto_examples/decomposition/plot_faces_decomposition.html
* その際、圧縮後の次元を増減させると、再構成された画像がどのように変化するか、調べよう。
* 注意: NMFで次元圧縮するときは、中心化する前の元の画像データを使いましょう。
  * NMFは、非負データを対象とした次元圧縮手法だからです。